In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from skimage.util import montage
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Input, Dense, Flatten
from keras.optimizers import SGD, Adam
from keras.callbacks import EarlyStopping
import os
import pdb
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

2022-12-08 09:43:43.066718: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
path = '../PKLot/PKLotSegmented_resized/'

In [3]:
# load train data
print('indexing train data')
train_img = []
train_label = []
for folder in os.listdir(path):
    if folder[0] != '.':
        for weather in os.listdir(os.path.join(path, folder)):
            if weather[0] != '.':
                for date in os.listdir(os.path.join(path, folder, weather))[:-4]:
                    if date[0] != '.':
                        for occupied in os.listdir(os.path.join(path, folder, weather, date)):
                            if occupied[0] != '.':
                                for img in os.listdir(os.path.join(path, folder, weather, date, occupied)):
                                    train_label.append(occupied)
                                    train_img.append(os.path.join(path, folder, weather, date, occupied, img))

# load test data
print('indexing test data')
test_img = []
test_label = []
for folder in os.listdir(path):
    if folder[0] != '.':
        for weather in os.listdir(os.path.join(path, folder)):
            if weather[0] != '.':
                for date in os.listdir(os.path.join(path, folder, weather))[-4:-2]:
                    if date[0] != '.':
                        for occupied in os.listdir(os.path.join(path, folder, weather, date)):
                            if occupied[0] != '.':
                                for img in os.listdir(os.path.join(path, folder, weather, date, occupied)):
                                    test_label.append(occupied)
                                    test_img.append(os.path.join(path, folder, weather, date, occupied, img))

# load validation data
print('indexing validation data')
valid_img = []
valid_label = []
for folder in os.listdir(path):
    if folder[0] != '.':
        for weather in os.listdir(os.path.join(path, folder)):
            if weather[0] != '.':
                for date in os.listdir(os.path.join(path, folder, weather))[-4:-2]:
                    if date[0] != '.':
                        for occupied in os.listdir(os.path.join(path, folder, weather, date)):
                            if occupied[0] != '.':
                                for img in os.listdir(os.path.join(path, folder, weather, date, occupied)):
                                    valid_label.append(occupied)
                                    valid_img.append(os.path.join(path, folder, weather, date, occupied, img))

indexing train data
indexing test data
indexing validation data


In [26]:
numoutputs = len(np.unique(train_label))
_, t = np.unique(train_label, return_inverse=True)
t_train = to_categorical(t, numoutputs)

_, t = np.unique(test_label, return_inverse=True)
t_test = to_categorical(t, numoutputs)

_, t = np.unique(valid_label, return_inverse=True)
t_valid = to_categorical(t, numoutputs)

In [29]:
# create dataframe to load with tf datagenerator
train_df = pd.DataFrame(list(zip(train_img, t_train)), columns =['img_path', 'label'])
test_df = pd.DataFrame(list(zip(test_img, t_test)), columns =['img_path', 'label'])
valid_df = pd.DataFrame(list(zip(valid_img, t_valid)), columns =['img_path', 'label'])

In [32]:
datagen = ImageDataGenerator(rescale=1./255.)

train_generator=datagen.flow_from_dataframe(
    dataframe=train_df,
    directory="./",
    x_col="img_path",
    y_col="label",
    #subset="training",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="raw",
    #target_size=(32,32),
    validate_filenames=False
    )
    
valid_generator=datagen.flow_from_dataframe(
    dataframe=valid_df,
    directory="./",
    x_col="img_path",
    y_col="label",
    #subset="training",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="raw",
    #target_size=(32,32),
    validate_filenames=False
    )

# test_datagen=ImageDataGenerator(rescale=1./255.)
test_generator=datagen.flow_from_dataframe(
    dataframe=test_df,
    directory="./",
    x_col="img_path",
    y_col="label",
    #subset="training",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="raw",
    #target_size=(32,32),
    validate_filenames=False
    )

Found 489852 non-validated image filenames.
Found 86714 non-validated image filenames.
Found 86714 non-validated image filenames.


In [ ]:
# create neural network
model = Sequential()
model.add(Input(Flatten()))
model.add(Dense(units=1000, activation='relu', name='hidden1'))
model.add(Dense(units=100, activation='sigmoid', name='hidden2'))
# model.add(Dense(units=50, activation='relu', name='hidden3'))
# model.add(Dense(units=50, activation='relu', name='hidden4'))
model.add(Dense(units=40, activation='softmax', name='output'))
model.summary()

In [ ]:
model.compile(
    optimizer= Adam(learning_rate=0.001),
    loss='mse',
    metrics=['accuracy']
    )
early_stop = EarlyStopping(
    monitor='accuracy', 
    patience=5, 
    min_delta=0, 
    mode='max'
    )

In [ ]:
model.fit_generator(generator=train_generator,
                    validation_data=valid_generator,
                    use_multiprocessing=True,
                    workers=4)